Aquí se importan torch, torchvision para manipular los datos, y matplotlib para la visualización.

In [1]:
import torch

from torchvision import datasets
from torchvision.transforms import ToTensor

import matplotlib.pyplot as plt

from torch.utils.data import DataLoader

import torch.nn as nn

from torch import optim

from torch.autograd import Variable

Utiliza una GPU ('cuda') si está disponible; de lo contrario, recurre a la CPU ('cpu'). ###

A diferencia de tf, aqui hay que espacificar que se tiene que usar la GPU, en tf se utiliza directamente la GPU si se detecta.

In [2]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
device

device(type='cuda')

Carga los datos de train y test en un nuevo directorio 'data'.

Es el equivalente a hacer en tf lo siguiente:



```
from tensorflow.keras.datasets import mnist
(train_images, train_labels), (test_images, test_labels) = mnist.load_data()
```





In [3]:
train_data = datasets.MNIST(
    root = 'data',
    train = True,
    transform = ToTensor(),
    download = True,
)
test_data = datasets.MNIST(
    root = 'data',
    train = False,
    transform = ToTensor()
)

100%|██████████| 9912422/9912422 [00:00<00:00, 421203270.73it/s]

Extracting data/MNIST/raw/train-images-idx3-ubyte.gz to data/MNIST/raw


100%|██████████| 28881/28881 [00:00<00:00, 58718222.89it/s]


Extracting data/MNIST/raw/train-labels-idx1-ubyte.gz to data/MNIST/raw



100%|██████████| 1648877/1648877 [00:00<00:00, 170838678.84it/s]


Extracting data/MNIST/raw/t10k-images-idx3-ubyte.gz to data/MNIST/raw



100%|██████████| 4542/4542 [00:00<00:00, 22255290.62it/s]

Extracting data/MNIST/raw/t10k-labels-idx1-ubyte.gz to data/MNIST/raw



Se utiliza '.size()' para ver las dimensiones de train y test.

In [4]:
print(train_data)
print(train_data.data.size())

Dataset MNIST
    Number of datapoints: 60000
    Root location: data
    Split: Train
    StandardTransform
Transform: ToTensor()
torch.Size([60000, 28, 28])


In [5]:
print(test_data)
print(test_data.data.size())

Dataset MNIST
    Number of datapoints: 10000
    Root location: data
    Split: Test
    StandardTransform
Transform: ToTensor()
torch.Size([10000, 28, 28])


Para cargar los datos se utiliza el '.DataLoader()', donde podemos especificar la cantidad de imagenes que habrá en cada lote. Además con num_workers se puede especificar la cantidad e subprocesos que se usarán para la carga de datos, a más 'trabajadores' mas velocidad al cargar los datos y mas carga en la CPU. Además una sobrecarga de 'trabajadores' puede llevar a un bloqueo de memoria.



In [8]:
loaders = {
    'train' : torch.utils.data.DataLoader(train_data,
                                          batch_size=100,
                                          shuffle=True,
                                          num_workers=1),

    'test'  : torch.utils.data.DataLoader(test_data,
                                          batch_size=100,
                                          shuffle=True,
                                          num_workers=1),
}
loaders

{'train': <torch.utils.data.dataloader.DataLoader at 0x7effa00fe860>,
 'test': <torch.utils.data.dataloader.DataLoader at 0x7effa00fcb50>}

Para poder hacer una red neuronal en pytorch tenemos que hacer una clase, esta debe de heredar de 'nn.Module', clase de la que derivan todas las redes neuronales de PyTorch. Con 'super(Modelo_CNN, self).__init__():' se lama al constructor de la clase padre.



---


Con 'nn.Conv2d()' se crea una capa conolucional 2D. En esta se especifica que la entrada tiene un canal (en este caso por se una escala de grises, si fuese una imagen RGB -> 'in_channels = 3'), se la aplicarán 16 filtros y que el tamaño del kernel es de 5*5.

Despues aplica la función de activación ReLU y hace un Maxpooling.

La siguiente capa es practicamente lo mismo, tan solo se cambia en número de filtros a 32.  

'Linear()' es una capa lineal que espera que sus entradas ya estén aplanadas, y el aplanamiento se maneja explícitamente en el método forward. Linear en PyTorch es lo mismo que '.Dense()' en keras. 10 es el numero de clases de salida que hay (numeros del 0 al 9). En este caso, hay 32 canales, cada uno de tamaño 7x7 (28/2-> 14/2-> 7), por lo tanto 32 * 7 * 7.

 32 * 7 * 7 representa el número total de elementos en el tensor aplanado que llegan a la capa lineal con 10 salidas



---

El método forward en una clase de red neuronal en PyTorch define cómo los datos de entrada (en este caso x) fluyen a través de la red.

'x' es el tensor de entrada que pasa por la capa conv1 y conv2. Antes de pasar 'x' a la capa lineal es necesario aplanarlo en un vector de una única dimensión.
'x.view(x.size(0), -1)' realiza esta operación. 'x.size(0)' es el tamaño del batch, y -1 le dice a PyTorch que calcule automáticamente la dimensión necesaria para que el aplanamiento sea correcto. En este caso, 32 * 7 * 7. Despues de esto, 'x' ya puede pasar por la capa .out para la que era necesario tener un vector aplanado.

Al final este método devuelve la predicción ('output') y el tensor 'x'.

In [20]:
class Modelo_CNN(nn.Module): # Define una nueva clase de red neuronal. Modelo_CNN
    def __init__(self):
      super(Modelo_CNN, self).__init__() # Llama al constructor de la clase padre para realizar la inicialización necesaria.
      # Primera capa convolucional
      self.conv1 = nn.Sequential(
          nn.Conv2d(
              in_channels=1,
              out_channels=16,
              kernel_size=5,
              stride=1,
              padding=2
          ),
          nn.ReLU(),
          nn.MaxPool2d(kernel_size=2)
      )
      self.conv2= nn.Sequential(
          nn.Conv2d(16, 32, 5, 1, 2),
          nn.ReLU(),
          nn.MaxPool2d(2)
      )
      self.out = nn.Linear(32 * 7 * 7, 10)

    def forward(self, x):
        x = self.conv1(x)
        x = self.conv2(x)
        x = x.view(x.size(0), -1)
        output = self.out(x)
        return output, x




Una vez está creado el modelo, podemos llamar a la función para que nos instancie un objeto 'Modelo_CNN()' y si hacemos un print pordemos ver las características del modelo

Es como el '.summary()'

In [21]:
mi_modelo = Modelo_CNN()
print(mi_modelo)

Modelo_CNN(
  (conv1): Sequential(
    (0): Conv2d(1, 16, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
    (1): ReLU()
    (2): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (conv2): Sequential(
    (0): Conv2d(16, 32, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
    (1): ReLU()
    (2): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (out): Linear(in_features=1568, out_features=10, bias=True)
)


Se epecifica la función de perdida y el optimizador.

In [22]:
loss_func = nn.CrossEntropyLoss()
loss_func

CrossEntropyLoss()

In [23]:
optimizer = optim.Adam(mi_modelo.parameters(), lr = 0.01)
optimizer

Adam (
Parameter Group 0
    amsgrad: False
    betas: (0.9, 0.999)
    capturable: False
    differentiable: False
    eps: 1e-08
    foreach: None
    fused: None
    lr: 0.01
    maximize: False
    weight_decay: 0
)

Para entrenar el modolo hay que hacer una función que se encargue de ello. Recibe el número de épocas que se va entrenar el modelo, el modelo a entrenar y un diccionario que contiene los DataLoaders para los conjuntos de datos de entrenamiento.

---

'cnn.train()' pone el modelo en modo entrenamiento, es importante para capas como dropout o batch normalization, que tienen comportamientos diferentes durante el entrenamiento y la evaluación.

Luego, en cada época el modelo itera sobre cada lote de datos (images y labels) en el DataLoader de entrenamiento.


'cnn(b_x)[0]' pasa el lote de imágenes a través del modelo y obtiene las salidas.
'loss_func(output, b_y)' calcula la pérdida comparando la salida del modelo con la etiqueta.



In [26]:
num_epochs = 5

def train(num_epochs, cnn, loaders):

    cnn.train()

    total_step = len(loaders['train'])

    for epoch in range(num_epochs):
        for i, (images, labels) in enumerate(loaders['train']):

            b_x = Variable(images)   # batch x
            b_y = Variable(labels)   # batch y
            output = cnn(b_x)[0]
            loss = loss_func(output, b_y)

            optimizer.zero_grad()   # limpia los gradientes antiguos
            loss.backward()         # calcula los gradientes
            optimizer.step()        # actualiza los parámetros

            if (i+1) % 100 == 0:
                print ('Epoch [{}/{}], Step [{}/{}], Loss: {:.4f}'.format(epoch + 1, num_epochs, i + 1, total_step, loss.item()))
            pass
        pass
    pass

train(num_epochs, mi_modelo, loaders)

Epoch [1/5], Step [100/600], Loss: 0.0654
Epoch [1/5], Step [200/600], Loss: 0.0664
Epoch [1/5], Step [300/600], Loss: 0.0027
Epoch [1/5], Step [400/600], Loss: 0.0813
Epoch [1/5], Step [500/600], Loss: 0.0059
Epoch [1/5], Step [600/600], Loss: 0.0078
Epoch [2/5], Step [100/600], Loss: 0.0258
Epoch [2/5], Step [200/600], Loss: 0.0555
Epoch [2/5], Step [300/600], Loss: 0.0134
Epoch [2/5], Step [400/600], Loss: 0.2154
Epoch [2/5], Step [500/600], Loss: 0.0691
Epoch [2/5], Step [600/600], Loss: 0.0214
Epoch [3/5], Step [100/600], Loss: 0.0279
Epoch [3/5], Step [200/600], Loss: 0.0936
Epoch [3/5], Step [300/600], Loss: 0.0029
Epoch [3/5], Step [400/600], Loss: 0.0735
Epoch [3/5], Step [500/600], Loss: 0.1277
Epoch [3/5], Step [600/600], Loss: 0.0709
Epoch [4/5], Step [100/600], Loss: 0.1238
Epoch [4/5], Step [200/600], Loss: 0.0762
Epoch [4/5], Step [300/600], Loss: 0.0220
Epoch [4/5], Step [400/600], Loss: 0.0519
Epoch [4/5], Step [500/600], Loss: 0.0266
Epoch [4/5], Step [600/600], Loss:

Esta función es la que comprobará la eficacia de la CNN.

In [29]:
def test(modelo, cargador_de_pruebas):
    # Poner el modelo en modo de evaluación
    modelo.eval()

    # Inicializar contadores de predicciones correctas y total de muestras
    correctas = 0
    total = 0

    # Desactivar el cálculo de gradientes
    with torch.no_grad():
        # Iterar sobre el conjunto de datos de prueba
        for imagenes, etiquetas in cargador_de_pruebas:
            # Realizar predicciones
            salidas, _ = modelo(imagenes)
            # Obtener la clase predicha para cada imagen
            _, predicciones = torch.max(salidas, 1)

            # Contar predicciones correctas y el total de muestras
            correctas += (predicciones == etiquetas).sum().item()
            total += etiquetas.size(0)

    # Calcular la precisión
    precision = 100 * correctas / total
    print(f'Precisión del modelo en el conjunto de datos de prueba: {precision:.2f}%')
    return precision

# Llamada a la función test
precision = test(mi_modelo, loaders['test'])


Precisión del modelo en el conjunto de datos de prueba: 97.71%
